In [1]:
import tensorflow as tf
import gym
import numpy as np
import copy
import time

## A2C

Actor-Critic Architecture

In [2]:
class A2C:
    def __init__(self, name : str, obs_space, action_space, sess):
        self.observation = obs_space
        self.action = action_space
        self.scope_name = name
        self.sess = sess
        
        with(tf.variable_scope(self.scope_name)):
            # placeholder for inputs to the network
            self.inputs = tf.placeholder(shape = [None] + list(self.observation.shape) , dtype = tf.float32)
        
            # build the two networks
            self.build_network()
            
            # stochastic action
            self.act = tf.multinomial(tf.log(self.act_probs),1)
            self.act = tf.reshape(self.act, shape = [-1])
            
    def build_network(self):
        # critic network gives out value prediction for the given inputs
        with tf.variable_scope('critic', reuse = tf.AUTO_REUSE):
            cout = tf.layers.dense(self.inputs, units = 16, activation = tf.tanh)
            cout = tf.layers.dense(cout, units = 32, activation = tf.tanh)
            self.value = tf.layers.dense(cout, units = 1, activation = None)
        
        # actor network spits out action probabilites
        with tf.variable_scope('actor', reuse = tf.AUTO_REUSE):
            aout = tf.layers.dense(self.inputs, 16,activation = tf.tanh)
            aout = tf.layers.dense(aout, units = 32, activation = tf.tanh)
            aout = tf.layers.dense(aout, units = 16, activation = tf.tanh)
            self.act_probs = tf.layers.dense(aout, self.action.n , activation = tf.nn.softmax)
                    
    # get actions based on the given inputs
    def get_action(self, inputs):
        return self.sess.run(self.act, feed_dict = {self.inputs : inputs})
    
    # get value prediction for the given inputs
    def get_value(self, inputs):
        return self.sess.run(self.value, feed_dict = {self.inputs : inputs})
    
    # get all trainable variables required for policy update later
    def trainable_vars(self):
        return tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,self.scope_name)

## PPO

In [3]:
class PPO:
    
    def __init__(self, env, sess, eps = 0.2, gamma = 0.95, clip1=1, clip2=0.01, learning_rate = 5e-5):
        self.sess = sess
        self.eps = eps
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.clip1 = clip1
        self.clip2 = clip2
        self.act_clip_max = 1
        self.act_clip_min = 1e-10
        
        self.pi = A2C("pi", env.observation_space, env.action_space, self.sess)
        self.old_pi = A2C("old_pi", env.observation_space, env.action_space, self.sess)
        
        self.pi_trainable_params = self.pi.trainable_vars()
        self.old_pi_trainable_params = self.old_pi.trainable_vars()
        
        with tf.variable_scope('update_policy'):
            self.update_ops = [old_pi_vals.assign(pi_vals) for pi_vals, old_pi_vals in zip(self.pi_trainable_params, self.old_pi_trainable_params)]
        
        with tf.variable_scope('training_inputs'):
            self.actions = tf.placeholder(shape = [None], dtype=tf.int32)
            self.rewards = tf.placeholder(shape = [None], dtype=tf.float32)
            self.v_next = tf.placeholder(shape = [None], dtype=tf.float32)
            self.adv = tf.placeholder(shape = [None], dtype=tf.float32)
            
        act_probs = self.hotify_action(self.pi.act_probs)
        act_old_probs = self.hotify_action(self.old_pi.act_probs)
            
        with tf.variable_scope("loss"):
        
            # loss calculations
            clipped_act_probs = tf.log(tf.clip_by_value(act_probs, self.act_clip_min, self.act_clip_max))
            clipped_old_act_probs = tf.log(tf.clip_by_value(act_old_probs, self.act_clip_min, self.act_clip_max))
            
            ratio = tf.exp(clipped_act_probs - clipped_old_act_probs)
    
            clipped_ratio = tf.clip_by_value(ratio, 1 -self.eps, 1 + self.eps)
            surrogate = tf.multiply(ratio, self.adv)
            surrogate_clipped = tf.multiply(clipped_ratio, self.adv)
            
            clipped_loss = tf.minimum(surrogate, surrogate_clipped)
            clipped_loss = tf.reduce_mean(clipped_loss)
            
            entropy = -tf.reduce_sum(self.pi.act_probs * tf.log(tf.clip_by_value(self.pi.act_probs, self.act_clip_min, self.act_clip_max)), 1)
            entropy = tf.reduce_mean(entropy, 0)
            
            value = self.pi.value
            error = self.rewards + self.gamma * self.v_next
            loss_value = tf.squared_difference(error, value)
            loss_value = tf.reduce_sum(loss_value)
            
            self.loss = -(clipped_loss - self.clip1 * loss_value + self.clip2 * entropy)
            self.loss_plot = tf.summary.scalar('loss', self.loss)
        
        opt = tf.train.AdamOptimizer(self.learning_rate, epsilon=1e-5)
        self.gradients = opt.compute_gradients(self.loss, var_list = self.pi_trainable_params)
        self.train_op = opt.minimize(self.loss, var_list = self.pi_trainable_params)

    # get action given state
    def get_action(self, inputs):
        return self.pi.get_action(inputs)

    # get value estimate given state
    def get_value(self, inputs):
        return self.pi.get_value(inputs)
    
    # update old policy network to the new network parameters    
    def update_old_policy(self):
        self.sess.run(self.update_ops)
    
    def train_policy(self, inputs, actions, rewards, v_next, advantages):
        self.sess.run(self.train_op, feed_dict = {self.pi.inputs : inputs,
                                                  self.old_pi.inputs : inputs,
                                                  self.actions: actions,
                                                  self.rewards: rewards,
                                                  self.v_next: v_next, 
                                                  self.adv: advantages})
    
    # get advantage estimates
    def get_gaes(self, rewards, v_preds, v_preds_next):
        deltas = [r_t + self.gamma * v_next - v for r_t, v_next, v in zip(rewards, v_preds_next, v_preds)]
        gaes = copy.deepcopy(deltas)
        for t in reversed(range(len(gaes) - 1)):
            gaes[t] = gaes[t] + self.gamma * gaes[t + 1]
        return gaes
    
    def hotify_action(self, action):
        action *= tf.one_hot(self.actions, action.shape[1])
        action = tf.reduce_sum(action, 1)
        return action
    
    def get_entropy(self, act_probs):
        entropy = -tf.reduce_sum(act_probs * tf.log(tf.clip_by_value(act_probs, self.act_clip_min, self.act_clip_max)), 1)
        return tf.reduce_mean(entropy, 0)        

In [4]:
def epoch_train(num_epochs, ppo, obs, actions, adv, rewards, v_preds_next):
    transitions = [obs, actions, adv, rewards, v_preds_next]
    
    for epochs in range(num_epochs):
        # random sampling
        index = indices = np.random.randint(0, obs.shape[0], size = 32)
        samples = [np.take(transition, index, axis=0) for transition in transitions]

        # training
        ppo.train_policy(inputs = samples[0],
                     actions = samples[1],
                     advantages = samples[2],
                     rewards = samples[3],
                     v_next = samples[4])

In [5]:
def preprocess(input):
    return np.stack([input])

In [6]:
def z_score(input):
    return (input - input.mean())/input.std()

## Hyperparameters

In [7]:
iterations = 7000
num_of_epochs = 6

## PPO runner

In [8]:
tf.reset_default_graph()  

with tf.Session() as sess:
    tensor_plot = tf.summary.FileWriter('log/ppo', graph = sess.graph)
    env = gym.make('CartPole-v0')
    ppo = PPO(env, sess)
    sess.run(tf.global_variables_initializer())
    state = env.reset()
    
    # save trained model
    saver = tf.train.Saver()
    
    for i in range(iterations+1):
        obs = []
        actions = []
        rewards = []
        values = []
        length = 0
        
        if i % 1000 == 0:
                print('Episode number: {}'.format(i))
             
        while True:
            length += 1
            
            state = preprocess(state)
            
            action = ppo.get_action(state)
            action = np.asscalar(action)
            
            value = ppo.get_value(state)
            value = np.asscalar(value)
            
            next_state, reward, done, _ = env.step(action)
            
            obs.append(state)
            actions.append(action)
            rewards.append(reward)
            values.append(value)
               
            if done:
                next_state = preprocess(next_state)
                next_value = ppo.get_value(next_state)
                next_value = np.asscalar(next_value)
                v_preds_next = values[1:] + [next_value]
                state = env.reset()
                break
            else:
                state = next_state
        
        tensor_plot.add_summary(tf.Summary(value = [tf.Summary.Value(tag="my_ppo_episode_rewards", simple_value = sum(rewards))]), i)
        tensor_plot.add_summary(tf.Summary(value = [tf.Summary.Value(tag="my_ppo_episode_length", simple_value = length)]), i)
        
        adv = ppo.get_gaes(rewards, values, v_preds_next)

        obs = np.reshape(obs, newshape=(-1,) + env.observation_space.shape)
        
        rewards = np.array(rewards)
        v_preds_next = np.array(v_preds_next)
        actions = np.array(actions)
        adv = z_score(np.array(adv))
                
        ppo.update_old_policy()

        epoch_train(num_of_epochs, ppo, obs, actions, adv, rewards, v_preds_next)
    
    path = saver.save(sess, "trained_model/ppo.ckpt")
    print("Model saved at {}".format(path))
    tensor_plot.close()
env.close()

print("Training complete. Check the tensorboard for plots")

/home/prasad/anaconda3/envs/tf/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


Episode number: 0
Episode number: 1000
Episode number: 2000
Episode number: 3000
Episode number: 4000
Episode number: 5000
Episode number: 6000
Episode number: 7000
Model saved at trained_model/ppo.ckpt
Training complete. Check the tensorboard for plots


## Extract Expert Trajectories

In [31]:
tf.reset_default_graph()
iterations = 20

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    env = gym.make('CartPole-v0')
    ppo = PPO(env, sess)
    
    saver = tf.train.Saver()
    
    saver.restore(sess, "trained_model/ppo.ckpt")
    print("Model restored!")
    
    state = env.reset()
    
    for itr in range(iterations):
        obs = []
        actions = []
        length = 0
        done = False
        
        while True:
            length += 1
            state = preprocess(state)
            
            action = ppo.get_action(state)
            action = np.asscalar(action)
            
            # take action
            next_state, reward, done, _ = env.step(action)
            
            obs.append(state)
            actions.append(action)
            
            if done:
                state = env.reset()
                break
            else:
                state = next_state
        
        obs = np.reshape(obs, newshape=[-1] + list(env.observation_space.shape))
        actions = np.array(actions)
        
        np.save("trajectories/expert_obs.npy", obs)
        np.save("trajectories/expert_actions.npy", actions)
        
env.close()    

INFO:tensorflow:Restoring parameters from trained_model/ppo.ckpt
Model restored!


# Render trained model (Demo)
## Condition for environment to be considered as solved:
Considered solved when the average reward is **greater than or equal to 195.0 over 100 consecutive trials**.
So, we check our test for the above condition and break the testing once this condition is satisfied.

Observe the rewards obtained in the demo on tensorboard under the tag *'test_episode_rewards'*

In [12]:
tf.reset_default_graph()
success_threshold = 195

with tf.Session() as sess:
    tensor_plot = tf.summary.FileWriter('log/ppo', graph = sess.graph)
    sess.run(tf.global_variables_initializer())
    env = gym.make('CartPole-v0')
    ppo = PPO(env, sess)
    
    saver = tf.train.Saver()
    succ_runs = 0
    
    saver.restore(sess, "trained_model/ppo.ckpt")
    print("Model restored!")
            
    state = env.reset()
    done = False
        
    for itr in range(501):
        rewards = []
        
        if itr % 10 == 0:
            print('Episode: {}'.format(itr))
            
        while True:
            state = preprocess(state)
            # if you want to render the solved environment consider uncommenting the time.sleep statement below to facilitate 
            # slow rendering 
            # env.render()

            action = ppo.get_action(state)
            action = np.asscalar(action)

            state,reward,done,_ = env.step(action)
            rewards.append(reward)
            
            # to demonstrate slow rendering
            # time.sleep(0.025)
            
            if done:
                state = env.reset()
                break
                
        tensor_plot.add_summary(tf.Summary(value = [tf.Summary.Value(tag="test_episode_rewards", simple_value = sum(rewards))]), itr) 
        
        if sum(rewards) >= success_threshold:
            succ_runs  += 1
            
            if succ_runs > 100:
                print("Solved at Episode: {}".format(itr))
                break
        else:
            succ_runs = 0
                    
env.close()

INFO:tensorflow:Restoring parameters from trained_model/ppo.ckpt
Model restored!
Episode: 0
Episode: 10
Episode: 20
Episode: 30
Episode: 40
Episode: 50
Episode: 60
Episode: 70
Episode: 80
Episode: 90
Episode: 100
Episode: 110
Episode: 120
Episode: 130
Episode: 140
Solved at Episode: 140


# Discriminator
Discriminator presensted by a Generative Adversarial Network to discriminate between the generated trajectory from the expert trajectory.

In [ ]:
class Discriminator:
    

## Baselines Implementation of PPO

From the stable-baselines repo to compare my implementation with the baselines implementation of PPO

In [2]:
from stable_baselines import PPO1

model = PPO1('MlpPolicy', 'CartPole-v0', verbose=1, tensorboard_log="log/ppo/")
model.learn(total_timesteps=10000)

Creating environment from the given name, wrapped in a DummyVecEnv.


/home/prasad/anaconda3/envs/tf/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


********** Iteration 0 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     -0.00178 |      -0.00693 |      78.77592 |       0.00012 |       0.69304
     -0.00702 |      -0.00692 |      76.62876 |       0.00090 |       0.69229
     -0.01035 |      -0.00690 |      74.69688 |       0.00298 |       0.69027
     -0.01465 |      -0.00687 |      72.93570 |       0.00662 |       0.68672
Evaluating losses...
     -0.01600 |      -0.00684 |      71.61264 |       0.00914 |       0.68429
----------------------------------
| EpLenMean       | 20.4         |
| EpRewMean       | 20.4         |
| EpThisIter      | 11           |
| EpisodesSoFar   | 11           |
| TimeElapsed     | 0.771        |
| TimestepsSoFar  | 256          |
| ev_tdlam_before | 0.0114       |
| loss_ent        | 0.684286     |
| loss_kl         | 0.009138748  |
| loss_pol_entpen | -0.00684286  |
| loss_pol_surr   | -0.016003426 |
| loss_vf_loss    | 71.61264     |
------

## Baselines Implementation of TRPO

From the stable-baselines repo to compare my implementation with the baselines implementation of TRPO

In [10]:
from stable_baselines import TRPO

model = TRPO('MlpPolicy', 'CartPole-v0', verbose=1, tensorboard_log="log/trpo/")
model.learn(total_timesteps=10000)

Creating environment from the given name, wrapped in a DummyVecEnv.
********** Iteration 0 ************
Optimizing Policy...
sampling
done in 0.518 seconds
computegrad
done in 0.109 seconds
cg
      iter residual norm  soln norm
         0     0.0187          0
         1   4.86e-05      0.104
         2   3.31e-05      0.346
         3   2.52e-05      0.363
         4   1.08e-07       0.49
         5   1.79e-06       0.49
         6    1.1e-09      0.492
         7   2.73e-11      0.492
done in 0.154 seconds
Expected: 0.019 Actual: 0.018
Stepsize OK!
vf
done in 0.076 seconds
---------------------------------
| EpLenMean       | 21.1        |
| EpRewMean       | 21.1        |
| EpThisIter      | 48          |
| EpisodesSoFar   | 48          |
| TimeElapsed     | 0.936       |
| TimestepsSoFar  | 1024        |
| entloss         | 0.0         |
| entropy         | 0.6851318   |
| ev_tdlam_before | -0.00177    |
| meankl          | 0.008057453 |
| optimgain       | 0.018265154 |
| surrgai

## Monitor Tensorboard for plots

1. Change directory into this project directory
2. Execute the following command
    `tensorboard --logdir=log/`
3. Visit the localhost page with the provided port number to monitor tensorboard